<a href="https://colab.research.google.com/github/whsu00/project/blob/master/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
#@title Setup
#@markdown Run me to clone the repo, install dependencies, and set up MuJoCo!

!rm -rf * .config
!git clone https://github.com/whsu00/project
!apt-get install -y libglu1-mesa-dev libgl1-mesa-dev libosmesa6-dev xvfb ffmpeg curl patchelf libglfw3 libglfw3-dev cmake zlib1g zlib1g-dev swig libosmesa6-dev libgl1-mesa-glx libglfw3
!pip install mujoco_py==2.0.2.8 mpi4py

!wget https://whsu00.com/mujoco.zip
!unzip mujoco.zip
!rm mujoco.zip
!mv .mujoco ~

MJC_PATH = "/root/.mujoco"
import os
os.environ['LD_LIBRARY_PATH'] += ':{}/mujoco200/bin'.format(MJC_PATH)
os.environ['MUJOCO_PY_MUJOCO_PATH'] = '{}/mujoco200'.format(MJC_PATH)
os.environ['MUJOCO_PY_MJKEY_PATH'] = '{}/mjkey.txt'.format(MJC_PATH)

!cp $MJC_PATH/mujoco200/bin/*.so /usr/lib/x86_64-linux-gnu/
%cd project

Cloning into 'project'...
remote: Enumerating objects: 122, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 253 (delta 70), reused 90 (delta 50), pack-reused 131
Receiving objects: 100% (253/253), 772.35 KiB | 10.88 MiB/s, done.
Resolving deltas: 100% (115/115), done.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libglu1-mesa-dev is already the newest version (9.0.0-2.1build1).
libglu1-mesa-dev set to manually installed.
zlib1g is already the newest version (1:1.2.11.dfsg-0ubuntu2).
zlib1g-dev is already the newest version (1:1.2.11.dfsg-0ubuntu2).
zlib1g-dev set to manually installed.
cmake is already the newest version (3.10.2-1ubuntu2.18.04.1).
curl is already the newest version (7.58.0-2ubuntu3.10).
libgl1-mesa-dev is already the newest version (20.0.8-0ubuntu1~18.04.1).
libgl1-mesa-dev set to manually installed.
ffmpeg is already the newest version (7:3.4.8-0ubunt

In [7]:
%load_ext autoreload
%autoreload 2
import utils, network, buffer, valor
import sys
import gym, torch
sys.path.insert(0, './utils/')

In [8]:
#@title Arguments

"""
from utils.run_utils import setup_logger_kwargs
logger_kwargs = setup_logger_kwargs(exp_name, seed)

valor(lambda: gym.make(env), actor_critic=ActorCritic, ac_kwargs=dict(hidden_dims=[hid]*l),
    disc=Discriminator, dc_kwargs=dict(hidden_dims=hid),
    gamma=gamma, seed=seed, episodes_per_epoch=episodes, epochs=epochs, logger_kwargs=logger_kwargs, con_dim=con)
""";

class Args:
    env = "HalfCheetah-v2" #@param {type:"string"}
    hid = 64 #@param {type:"integer"}
    l = 2 #@param {type:"integer"}
    exp_name = "valor" #@param {type:"string"}
    seed = 0 #@param {type:"integer"}
    episodes_per_epoch = 40 #@param {type:"integer"}
    epochs = 50 #@param {type:"integer"}
    gamma = 0.99 #@param {type:"number"}
    pi_lr = 3e-4 #@param {type:"number"}
    vf_lr = 1e-3 #@param {type:"number"}
    dc_lr = 2e-3 #@param {type:"number"}
    train_v_iters = 80 #@param {type:"integer"}
    train_dc_iters = 50 #@param {type:"integer"}
    train_dc_interv = 2 #@param {type:"integer"}
    lam = 0.97 #@param {type:"number"}
    max_ep_len = 1000 #@param {type:"integer"}
    context_dim = 4 #@param {type:"integer"}
    max_context_dim = 64 #@param {type:"integer"}
    save_freq = 10 #@param {type:"integer"}
    k = 1 #@param {type:"integer"}
    lstmFlag = False #@param{type:"boolean"}

      
    env_fn = (lambda e: lambda self: gym.make(e))(env)
    actor_critic = network.ActorCritic
    ac_kwargs = {
        "hidden_dims": [hid]*l, 
        "lstmFlag": lstmFlag
    }
    disc = network.Discriminator
    dc_kwargs = {
        "hidden_dims": hid
    }

    empty_flag = ()
    def get(self, key, default=empty_flag):
        if hasattr(self, key):
            return getattr(self, key)
        elif default is not Args.empty_flag:
            return default
        raise KeyError("Args object does not have this attribute defined!")

    def __getitem__(self, key):
        return self.get(key)

    def __contains__(self, key):
        return hasattr(self, key)

    def __setitem__(self, key, value):
        setattr(self, key, value)

    


NameError: ignored

In [ ]:
from run_utils import setup_logger_kwargs
args = Args()
args.logger_kwargs = setup_logger_kwargs(args.exp_name, args.seed)

In [ ]:
valor.valor(args)

Using GPU id 0
Logging data to /content/project/data/valor/valor_s0/progress.txt
Saving config:

{
    "ac_kwargs":	{},
    "actor_critic":	"ActorCritic",
    "args":	{
        "<__main__.Args object at 0x7fe36e97e160>":	{
            "logger_kwargs":	{
                "exp_name":	"valor",
                "output_dir":	"/content/project/data/valor/valor_s0"
            }
        }
    },
    "context_dim":	4,
    "dc_kwargs":	{},
    "dc_lr":	0.002,
    "disc":	"Discriminator",
    "env_fn":	"<bound method Args.<lambda>.<locals>.<lambda> of <__main__.Args object at 0x7fe36e97e160>>",
    "episodes_per_epoch":	40,
    "epochs":	50,
    "exp_name":	"valor",
    "gamma":	0.99,
    "k":	1,
    "lam":	0.97,
    "logger":	{
        "<logx.EpochLogger object at 0x7fe36e90cc50>":	{
            "epoch_dict":	{},
            "exp_name":	"valor",
            "first_row":	true,
            "log_current_row":	{},
            "log_headers":	[],
            "output_dir":	"/content/project/data/valor/